In [58]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from geopandas_view import view 


In [59]:
prices = []
beds = []
baths = []
sizes = []
addresses = []


In [63]:
import undetected_chromedriver as uc
import time
options.add_argument("--enable-javascript")
options.add_argument("start-maximized")
options = uc.ChromeOptions()
driver = uc.Chrome(options = options, browser_executable_path="/usr/bin/chromium", driver_executable_path="/tmp/chromedriver")
driver.get('https://www.realtor.com/realestateandhomes-search/Portland_OR')
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(20)  # Wait for 2 seconds


In [64]:
content = driver.page_source
soup = BeautifulSoup(content, features='html.parser')


In [65]:
print(soup)

<html lang="en"><head><script async="" src="https://bat.bing.com/p/insights/s/0.7.32"></script><script async="" src="https://bat.bing.com/p/insights/t/25046895"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=AW-1049445407" type="text/javascript"></script><script async="" src="https://www.google-analytics.com/analytics.js" type="text/javascript"></script><script async="" src="https://bat.bing.com/bat.js" type="text/javascript"></script><script async="" src="https://cdn.segment.com/next-integrations/integrations/vendor/commons.c42222c4cb2f8913500f.js.gz" status="loaded" type="text/javascript"></script><script async="" src="https://cdn.segment.com/next-integrations/integrations/google-adwords-new/1.3.0/google-adwords-new.dynamic.js.gz" status="loaded" type="text/javascript"></script><script async="" src="https://cdn.segment.com/next-integrations/integrations/bing-ads/2.0.1/bing-ads.dynamic.js.gz" status="loaded" type="text/javascript"></script><script async="" 

In [93]:
import pandas as pd

property_cards = soup.select('div[class^="CardContent__StyledCardContent"]')

# Create empty lists to store data
prices = []
bedrooms = []
bathrooms = []
sqfts = []
addresses = []
zip_codes = []

for card in property_cards:
    # Initialize variables with None in case the element is not found
    price = None
    bedroom_raw = None
    bathroom_raw = None
    sqft_raw = None
    
    # Find price element
    price_elem = card.find('div', class_='card-price')
    if price_elem:
        price_text = price_elem.text.strip()
        price = ''.join(filter(str.isdigit, price_text))  # Extract only the numerical part
        # Strip the $ sign
        price = price.lstrip('$')
    
    # Find bedroom element
    bedroom_elem = card.find('li', class_='PropertyBedMetastyles__StyledPropertyBedMeta-rui__a4nnof-0')
    if bedroom_elem:
        bedroom_raw = bedroom_elem.text.strip()
    
    # Find bathroom element
    bathroom_elem = card.find('li', class_='PropertyBathMetastyles__StyledPropertyBathMeta-rui__sc-67m6bo-0')
    if bathroom_elem:
        bathroom_raw = bathroom_elem.text.strip()
    
    # Find sqft element
    sqft_elem = card.find('li', class_='PropertySqftMetastyles__StyledPropertySqftMeta-rui__sc-1gdau7i-0')
    if sqft_elem:
        sqft_raw = sqft_elem.text.strip()
    
    # Clean up bedroom, bathroom, and sqft
    bedroom = ''.join(filter(str.isdigit, bedroom_raw)) if bedroom_raw else None
    bathroom = ''.join(filter(str.isdigit, bathroom_raw)) if bathroom_raw else None
    sqft = ''.join(filter(str.isdigit, sqft_raw.split()[0]))[:4] if sqft_raw else None
    
    # Find address element within the current card and extract ZIP code
    address_elem = card.find('div', class_='card-address')
    address = address_elem.text.strip() if address_elem else None
    zip_code = address.split()[-1] if address else None
    
    # Append data to lists
    prices.append(price)
    bedrooms.append(bedroom)
    bathrooms.append(bathroom)
    sqfts.append(sqft)
    addresses.append(address)
    zip_codes.append(zip_code)

# Create DataFrame
df = pd.DataFrame({
    'Price': prices,
    'Bedrooms': bedrooms,
    'Bathrooms': bathrooms,
    'Sqft': sqfts,
    'Address': addresses,
    'ZIP Code': zip_codes
})

# Save DataFrame to CSV
df.to_csv('listings_oregon.csv', index=False)


AttributeError: 'NoneType' object has no attribute 'text'

In [81]:
import pandas as pd
import requests
import json
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
df = pd.read_csv("listings.csv")


In [ ]:
# Import geocoding tool
from geopandas.tools import geocode

# Geocode addresses using Nominatim. Remember to provide a custom "application name" in the user_agent parameter!
geo = geocode(df['Address'], provider='nominatim', user_agent='autogis_ps', timeout=3)

In [ ]:
filtered_geo = gpd.GeoDataFrame(geo).where(lambda x : x!=None).dropna()
filtered_geo

In [ ]:
view(filtered_geo)